# Spaceship Titanic Competition in kaggle

In [1]:
! unzip /content/spaceship-titanic.zip

Archive:  /content/spaceship-titanic.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [85]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')
df_test['Transported'] = False
df = pd.concat([df_train, df_test],sort=False)
df.drop(['Name','PassengerId'],axis=1 , inplace=True)

In [86]:
df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [87]:
df.shape[0] == df_train.shape[0] + df_test.shape[0]

True

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12970 entries, 0 to 4276
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12682 non-null  object 
 1   CryoSleep     12660 non-null  object 
 2   Cabin         12671 non-null  object 
 3   Destination   12696 non-null  object 
 4   Age           12700 non-null  float64
 5   VIP           12674 non-null  object 
 6   RoomService   12707 non-null  float64
 7   FoodCourt     12681 non-null  float64
 8   ShoppingMall  12664 non-null  float64
 9   Spa           12686 non-null  float64
 10  VRDeck        12702 non-null  float64
 11  Transported   12970 non-null  bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 1.2+ MB


In [89]:
df.isna().sum()

,0
HomePlanet,288
CryoSleep,310
Cabin,299
Destination,274
Age,270
VIP,296
RoomService,263
FoodCourt,289
ShoppingMall,306
Spa,284


In [90]:
df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


in column Cabin we need to sepaate them into three column by doing some featues engineering

In [91]:
df[['Deck','Num','Side']] = df['Cabin'].str.split('/',expand=True)
df.drop('Cabin',axis=1,inplace=True)
df['Deck'] = df['Deck'].fillna('U')

In [92]:
df.Deck.value_counts()

,count
Deck,
F,4239
G,3781
E,1323
B,1141
C,1102
D,720
A,354
U,299
T,11


In [93]:
df.isna().sum()[df.isna().sum()>0]

,0
HomePlanet,288
CryoSleep,310
Destination,274
Age,270
VIP,296
RoomService,263
FoodCourt,289
ShoppingMall,306
Spa,284
VRDeck,268


In [94]:
df['Num']= df['Num'].fillna(-1)
df['Side'] = df['Side'].fillna('U')

In [95]:
df.isna().sum()[df.isna().sum()>0]

,0
HomePlanet,288
CryoSleep,310
Destination,274
Age,270
VIP,296
RoomService,263
FoodCourt,289
ShoppingMall,306
Spa,284
VRDeck,268


In [96]:
df.Deck.value_counts()

,count
Deck,
F,4239
G,3781
E,1323
B,1141
C,1102
D,720
A,354
U,299
T,11


In [97]:
df.Num.value_counts()

,count
Num,
-1,299
82,34
56,28
4,28
230,27
...,...
1544,1
1623,1
1835,1


In [98]:
df.Side.value_counts()

,count
Side,
S,6381
P,6290
U,299


In [99]:
# encoding label
df['Deck']=df['Deck'].map({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7,'U':8})
df['Side']=df['Side'].map({'U':-1,'P':1,'S':2})

In [100]:
df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,1,0,1
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,5,0,2
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,0,2
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,0,2
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,5,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,False,6,1496,2
4273,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,False,8,-1,-1
4274,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,False,3,296,1
4275,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,False,3,297,1


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12970 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12682 non-null  object 
 1   CryoSleep     12660 non-null  object 
 2   Destination   12696 non-null  object 
 3   Age           12700 non-null  float64
 4   VIP           12674 non-null  object 
 5   RoomService   12707 non-null  float64
 6   FoodCourt     12681 non-null  float64
 7   ShoppingMall  12664 non-null  float64
 8   Spa           12686 non-null  float64
 9   VRDeck        12702 non-null  float64
 10  Transported   12970 non-null  bool   
 11  Deck          12970 non-null  int64  
 12  Num           12970 non-null  object 
 13  Side          12970 non-null  int64  
dtypes: bool(1), float64(6), int64(2), object(5)
memory usage: 1.4+ MB


In [102]:
from sklearn.impute import KNNImputer
impute_lis = ['Age', 'VIP', 'Num', 'CryoSleep', 'Side', 'Deck', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
rest = list(set(df.columns) - set(impute_lis))
df_rest = df[rest]
imp = KNNImputer()
df_imputed = imp.fit_transform(df[impute_lis])
df_imputed = pd.DataFrame(df_imputed, columns = impute_lis)
df = pd.concat([df_rest.reset_index(drop = True), df_imputed.reset_index(drop = True)], axis = 1)

In [103]:
df.isna().sum()

,0
HomePlanet,288
Destination,274
Transported,0
Age,0
VIP,0
Num,0
CryoSleep,0
Side,0
Deck,0
RoomService,0


In [104]:
df['HomePlanet'] = df['HomePlanet'].fillna('U')
df['Destination'] = df['Destination'].fillna('U')
category_colls = ['HomePlanet', 'Destination']

for col in category_colls:
    df = pd.concat([df, pd.get_dummies(df[col], prefix = col)], axis = 1)

In [105]:
df = df.drop(columns = category_colls)

In [106]:
df.isna().sum()

,0
Transported,0
Age,0
VIP,0
Num,0
CryoSleep,0
Side,0
Deck,0
RoomService,0
FoodCourt,0
ShoppingMall,0


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Transported                12970 non-null  bool   
 1   Age                        12970 non-null  float64
 2   VIP                        12970 non-null  float64
 3   Num                        12970 non-null  float64
 4   CryoSleep                  12970 non-null  float64
 5   Side                       12970 non-null  float64
 6   Deck                       12970 non-null  float64
 7   RoomService                12970 non-null  float64
 8   FoodCourt                  12970 non-null  float64
 9   ShoppingMall               12970 non-null  float64
 10  Spa                        12970 non-null  float64
 11  VRDeck                     12970 non-null  float64
 12  HomePlanet_Earth           12970 non-null  bool   
 13  HomePlanet_Europa          12970 non-null  boo

In [108]:
# feature Engineering
bill_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df['amt_spent'] = df[bill_cols].sum(axis = 1)
df['std_amt_spent'] = df[bill_cols].std(axis = 1)
df['mean_amt_spent'] = df[bill_cols].mean(axis = 1)

df['3_high_cols'] = df['CryoSleep'] + df['HomePlanet_Europa'] + df['Destination_55 Cancri e']
df['3_low_cols'] = df['mean_amt_spent'] + df['amt_spent'] + df['HomePlanet_Earth']

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Transported                12970 non-null  bool   
 1   Age                        12970 non-null  float64
 2   VIP                        12970 non-null  float64
 3   Num                        12970 non-null  float64
 4   CryoSleep                  12970 non-null  float64
 5   Side                       12970 non-null  float64
 6   Deck                       12970 non-null  float64
 7   RoomService                12970 non-null  float64
 8   FoodCourt                  12970 non-null  float64
 9   ShoppingMall               12970 non-null  float64
 10  Spa                        12970 non-null  float64
 11  VRDeck                     12970 non-null  float64
 12  HomePlanet_Earth           12970 non-null  bool   
 13  HomePlanet_Europa          12970 non-null  boo

In [112]:
df.corr()['Transported'].sort_values(ascending = False)

,Transported
Transported,1.000000
CryoSleep,0.324373
3_high_cols,0.284177
HomePlanet_Europa,0.131977
Destination_55 Cancri e,0.083625
Side,0.059872
FoodCourt,0.034746
HomePlanet_U,0.006403
HomePlanet_Mars,0.005643
ShoppingMall,0.004154


In [113]:
df.Transported.value_counts()

,count
Transported,
False,8592
True,4378


## Modeling

In [127]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier , GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report , precision_score , recall_score , f1_score , accuracy_score , confusion_matrix

In [115]:
df_train = df[:df_train.shape[0]]
df_test = df[df_train.shape[0]:]
df_test.drop('Transported',axis=1,inplace=True)
df_train.shape , df_test.shape

<ipython-input-115-9fb0b8e7c210>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop('Transported',axis=1,inplace=True)


((8693, 25), (4277, 24))

In [116]:
df_train.Transported.value_counts()

,count
Transported,
True,4378
False,4315


In [131]:
## Split data
X = df_train.drop('Transported',axis=1)
y = df_train['Transported']

X_tarin , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [120]:
model_1 = XGBClassifier()
model_1.fit(X_tarin,y_train)
y_pred = model_1.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.79      0.79      0.79       861
        True       0.79      0.79      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



In [121]:
model_2 = DecisionTreeClassifier()
model_2.fit(X_tarin,y_train)
y_pred = model_2.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.76      0.75      0.75       861
        True       0.75      0.76      0.76       878

    accuracy                           0.76      1739
   macro avg       0.76      0.76      0.76      1739
weighted avg       0.76      0.76      0.76      1739



In [122]:
model_3 = RandomForestClassifier()
model_3.fit(X_tarin,y_train)
y_pred = model_3.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.78      0.81      0.79       861
        True       0.81      0.77      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



In [123]:
model_4 = AdaBoostClassifier()
model_4.fit(X_tarin,y_train)
y_pred = model_4.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.79      0.73      0.76       861
        True       0.76      0.81      0.78       878

    accuracy                           0.77      1739
   macro avg       0.77      0.77      0.77      1739
weighted avg       0.77      0.77      0.77      1739



In [124]:
model_5 = GradientBoostingClassifier()
model_5.fit(X_tarin,y_train)
y_pred = model_5.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.81      0.75      0.78       861
        True       0.77      0.83      0.80       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



In [125]:
model_6 = LGBMClassifier()
model_6.fit(X_tarin,y_train)
y_pred = model_6.predict(X_test)
print(classification_report(y_test,y_pred))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001417 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2705
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230
              precision    recall  f1-score   support

       False       0.81      0.78      0.80       861
        True       0.79      0.82      0.81       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



In [128]:
model_7 = LogisticRegression()
model_7.fit(X_tarin,y_train)
y_pred = model_7.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.80      0.73      0.76       861
        True       0.75      0.83      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##HyperParameter

In [133]:
# XGboost Classifer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score  # Or any other relevant metric

# Assuming you have your data in X_train, y_train

# Create an XGBoost classifier object


# Define the hyperparameter grid to search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

# Create a GridSearchCV object
grid_search = GridSearchCV(
    estimator=model_1,
    param_grid=param_grid,
    scoring='accuracy',  # Use your desired scoring metric
    cv=5,  # Number of cross-validation folds
    n_jobs=-1,  # Use all available CPU cores
    verbose=2,  # Print detailed output during the search
)

# Fit the GridSearchCV object to your training data
grid_search.fit(X_tarin, y_train)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

# Get the best model
best_xgb_model = grid_search.best_estimator_

# Make predictions on the test set (if you have one)
y_pred = best_xgb_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best hyperparameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.9}
Accuracy: 0.7941345600920069


In [137]:
opt_mode1 = XGBClassifier(colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3, n_estimators= 300, subsample= 0.9)
opt_mode1.fit(X_tarin,y_train)
y_pred = opt_mode1.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.80      0.77      0.79       861
        True       0.79      0.81      0.80       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



In [138]:
# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 20],  # Include None for unlimited depth
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(
    estimator=model_3,  # Assuming model_3 is your RandomForestClassifier
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=2,
)

# Fit the GridSearchCV object to your training data
grid_search.fit(X_tarin, y_train)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

# Get the best model
best_rf_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_rf_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred)) # Added classification_report

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
65 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py",

Best hyperparameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Accuracy: 0.7918343875790684
              precision    recall  f1-score   support

       False       0.79      0.79      0.79       861
        True       0.79      0.80      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



In [142]:
test_data = pd.read_csv('/content/test.csv')
pred = model_3.predict(df_test)

submission = pd.DataFrame()
submission['PassengerId'] = test_data['PassengerId']
submission['Transported'] = pred
submission.to_csv('final_submission_SpaceShip_Titanic.csv', index=False)